In [2]:
import numpy as np
import pandas as pd

In [3]:
file_t = open('jester-data-1.csv').readlines()

In [6]:
mp = pd.read_csv('jester-data-1.csv', header=None).as_matrix()

In [7]:
mp

array([[ 74.  ,  -7.82,   8.79, ...,  99.  ,  99.  ,  99.  ],
       [100.  ,   4.08,  -0.29, ...,   0.34,  -4.32,   1.07],
       [ 49.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       ...,
       [ 39.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       [ 37.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       [ 72.  ,   2.43,   2.67, ...,  99.  ,  99.  ,  99.  ]])

In [15]:
def get_matrix():
    return pd.read_csv('jester-data-1.csv', header=None, na_values=99).iloc[:,1:].as_matrix()

In [24]:
class JokerObject:
    def __init__(self, matrix):
        self.matrix = matrix
        self.item_means = np.nanmean(matrix, 0)
        self.user_means = np.nanmean(matrix, 1)
    def get_matrix(self):
        return self.matrix
    def get_items_m(self):
        return self.item_means
    def get_item_m(self, key):
        return self.item_means[key]
    def get_users_m(self):
        return self.user_means
    def get_user_m(self, key):
        return self.user_means[key]